In [1]:
import pandas as pd
import requests
import json
import urllib.request
import time
from bs4 import BeautifulSoup
import re
import numpy as np

In [2]:
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize

In [3]:
data = pd.read_csv("../Firefox/final_firefox_data_15thmarch.csv")

In [4]:
data.head()

,Bug ID,Type,Summary,Product,Component,Status,Resolution,Changed,Description,Duplicate_Bug_Ids
0,1399936,defect,Shield unenrolls users in active preference ex...,Firefox,Normandy Client,VERIFIED,FIXED,2018-01-27 11:10:40,the activity stream team is conducting a study...,NaN
1,1387208,defect,browser_as_load_location.js is permafailing on...,Firefox,New Tab Page,RESOLVED,FIXED,2019-06-28 14:29:46,filed by rvandermeulen at mozilla com j...,NaN
2,1368432,defect,web.skype.com is inaccessible: SEC_ERROR_OCSP_...,Firefox,Untriaged,RESOLVED,DUPLICATE,2017-05-29 04:49:00,user agent mozilla gecko firefox ...,1368433.0
3,1383741,defect,Perma failure when 56 merge to beta in browser...,Firefox,Preferences,RESOLVED,FIXED,2017-07-27 02:02:17,found via uplift simulations perma failure w...,NaN
4,1388753,defect,Bookmarks and history pages in Library sub men...,Firefox,Toolbars and Customization,VERIFIED,FIXED,2017-09-22 10:09:03,user agent mozilla gecko firefox ...,NaN


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43159 entries, 0 to 43158
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Bug ID             43159 non-null  int64  
 1   Type               38278 non-null  object 
 2   Summary            38474 non-null  object 
 3   Product            38474 non-null  object 
 4   Component          38474 non-null  object 
 5   Status             38474 non-null  object 
 6   Resolution         38451 non-null  object 
 7   Changed            38474 non-null  object 
 8   Description        43159 non-null  object 
 9   Duplicate_Bug_Ids  7847 non-null   float64
dtypes: float64(1), int64(1), object(8)
memory usage: 3.3+ MB


In [6]:
sentences = data['Description']

In [7]:
tokens = []
for s in sentences:
    cleanedTex=re.sub(r'[^\w\s]','',str(s)).lower()
    words = (word_tokenize(cleanedTex))
    tokens .append(words)

In [8]:
bm25 = BM25Okapi(tokens)

In [9]:
data = data.rename(columns = {"Bug ID":"Bug_Id"})

In [10]:
data = data.reset_index(drop=True)

In [11]:
data['Description']=data['Description'].values.astype('object')

# On complete data

In [12]:
count=0
dup_indices_with_bug_id = {}
similarity_scores_before = []
similarity_scores = {}
issues = data.Bug_Id

for i, row in data.iterrows():
    
    issue_id = row.Bug_Id
    dup_id = row.Duplicate_Bug_Ids
    
    if dup_id not in issues.values:
        count+=1
#         print(f'{i}. Duplicate ID {dup_id} not found in issue IDs')
        continue

    similarity_row = bm25.get_scores(tokens[i])
    argsort_similarity = np.argsort(similarity_row)[::-1][1:]
    similarity_scores_before = (np.sort(similarity_row)[::-1][1:])
#     similarity_scores[issue_id] = np.min(similarity_scores_before[np.nonzero(similarity_scores_before)])
    dup_bug_index = list(issues[issues == dup_id].index)[0]
    dup_ranking = np.argmax(argsort_similarity == np.int64(dup_bug_index))
    
    dup_indices_with_bug_id[issue_id] = dup_ranking
# print(count)

In [13]:
count_at_1=0
count_at_5=0
count_at_10=0
count_at_20=0
count_at_25=0
count_at_30=0
count_at_50=0
count_at_75=0
count_at_100=0

for key, value in enumerate(dup_indices_with_bug_id.values()):
    if(value < 1):
        count_at_1+=1
    if(value < 5):
        count_at_5+=1
    if(value < 10):
        count_at_10+=1
    if(value < 20):
        count_at_20+=1
    if(value < 25):
        count_at_25+=1
    if(value < 30):
        count_at_30+=1
    if(value < 50):
        count_at_50+=1
    if(value < 75):
        count_at_75+=1
    if(value < 100):
        count_at_100+=1
        
print("count at 1",count_at_1)        
print("count at 5",count_at_5)
print("count at 10",count_at_10)
print("count at 20",count_at_20)
print("count at 25",count_at_25)
print("count at 30",count_at_30)
print("count at 50",count_at_50)
print("count at 75",count_at_75)
print("count at 100",count_at_100)

N = len(dup_indices_with_bug_id)

recall_rate_at_1 = count_at_1/N
print("recall_rate_at_1",recall_rate_at_1)

recall_rate_at_5 = count_at_5/N
print("recall_rate_at_5",recall_rate_at_5)

recall_rate_at_10 = count_at_10/N
print("recall_rate_at_10",recall_rate_at_10)

recall_rate_at_20 = count_at_20/N
print("recall_rate_at_20",recall_rate_at_20)

recall_rate_at_25 = count_at_25/N
print("recall_rate_at_25",recall_rate_at_25)

recall_rate_at_30 = count_at_30/N
print("recall_rate_at_30",recall_rate_at_30)

recall_rate_at_50 = count_at_50/N
print("recall_rate_at_50",recall_rate_at_50)

recall_rate_at_75 = count_at_75/N
print("recall_rate_at_75",recall_rate_at_75)

recall_rate_at_100 = count_at_100/N
print("recall_rate_at_10",recall_rate_at_100)

count at 1 703
count at 5 1540
count at 10 1891
count at 20 2254
count at 25 2361
count at 30 2458
count at 50 2803
count at 75 3052
count at 100 3230
recall_rate_at_1 0.09114482043303514
recall_rate_at_5 0.19966290678075976
recall_rate_at_10 0.24517049137819266
recall_rate_at_20 0.2922338908336575
recall_rate_at_25 0.3061065733177752
recall_rate_at_30 0.31868274342019964
recall_rate_at_50 0.3634124205886166
recall_rate_at_75 0.3956955788927784
recall_rate_at_10 0.4187734992869182


# Textually Similar

In [14]:
textually_similar_data = pd.read_csv("../Firefox/firefox_textually_similar_duplicates.csv")

In [15]:
textually_similar_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3923 entries, 0 to 3922
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Bug ID             3923 non-null   int64  
 1   Type               3923 non-null   object 
 2   Summary            3923 non-null   object 
 3   Product            3923 non-null   object 
 4   Component          3923 non-null   object 
 5   Status             3923 non-null   object 
 6   Resolution         3923 non-null   object 
 7   Changed            3923 non-null   object 
 8   Description        3923 non-null   object 
 9   Duplicate_Bug_Ids  3923 non-null   float64
dtypes: float64(1), int64(1), object(8)
memory usage: 306.6+ KB


In [16]:
sentence_sim = textually_similar_data.Description

tokens_sim = []
for s in sentence_sim:
    cleanedTex=re.sub(r'[^\w\s]','',str(s)).lower()
    words = (word_tokenize(cleanedTex))
    tokens_sim.append(words)

In [19]:
count=0
dup_indices_with_bug_id_similar = {}
similarity_scores_before = []
similarity_scores = {}
issues = data.Bug_Id

for i, row in textually_similar_data.iterrows():
    
    issue_id = row["Bug ID"]
    dup_id = row.Duplicate_Bug_Ids
    
    if dup_id not in issues.values:
        count+=1
#         print(f'{i}. Duplicate ID {dup_id} not found in issue IDs')
        continue
    if i==4427:
        break
    similarity_row = bm25.get_scores(tokens_sim[i])
    argsort_similarity = np.argsort(similarity_row)[::-1][1:]
#     similarity_scores_before = (np.sort(similarity_row)[::-1][1:])
#     similarity_scores[issue_id] = np.min(similarity_scores_before[np.nonzero(similarity_scores_before)])
    dup_bug_index = list(issues[issues == dup_id].index)[0]
    dup_ranking = np.argmax(argsort_similarity == np.int64(dup_bug_index))
    
    dup_indices_with_bug_id_similar[issue_id] = dup_ranking
# print(count)

In [20]:
count_at_1=0
count_at_5=0
count_at_10=0
count_at_20=0
count_at_25=0
count_at_30=0
count_at_50=0
count_at_75=0
count_at_100=0

for key, value in enumerate(dup_indices_with_bug_id_similar.values()):
    if(value < 1):
        count_at_1+=1
    if(value < 5):
        count_at_5+=1
    if(value < 10):
        count_at_10+=1
    if(value < 20):
        count_at_20+=1
    if(value < 25):
        count_at_25+=1
    if(value < 30):
        count_at_30+=1
    if(value < 50):
        count_at_50+=1
    if(value < 75):
        count_at_75+=1
    if(value < 100):
        count_at_100+=1
        
print("count at 1",count_at_1)        
print("count at 5",count_at_5)
print("count at 10",count_at_10)
print("count at 20",count_at_20)
print("count at 25",count_at_25)
print("count at 30",count_at_30)
print("count at 50",count_at_50)
print("count at 75",count_at_75)
print("count at 100",count_at_100)

N = len(dup_indices_with_bug_id_similar)

recall_rate_at_1 = count_at_1/N
print("recall_rate_at_1",recall_rate_at_1)

recall_rate_at_5 = count_at_5/N
print("recall_rate_at_5",recall_rate_at_5)

recall_rate_at_10 = count_at_10/N
print("recall_rate_at_10",recall_rate_at_10)

recall_rate_at_20 = count_at_20/N
print("recall_rate_at_20",recall_rate_at_20)

recall_rate_at_25 = count_at_25/N
print("recall_rate_at_25",recall_rate_at_25)

recall_rate_at_30 = count_at_30/N
print("recall_rate_at_30",recall_rate_at_30)

recall_rate_at_50 = count_at_50/N
print("recall_rate_at_50",recall_rate_at_50)

recall_rate_at_75 = count_at_75/N
print("recall_rate_at_75",recall_rate_at_75)

recall_rate_at_100 = count_at_100/N
print("recall_rate_at_10",recall_rate_at_100)

count at 1 425
count at 5 889
count at 10 1089
count at 20 1265
count at 25 1310
count at 30 1351
count at 50 1540
count at 75 1646
count at 100 1716
recall_rate_at_1 0.11087920688755544
recall_rate_at_5 0.23193321158361596
recall_rate_at_10 0.28411166188364206
recall_rate_at_20 0.330028698147665
recall_rate_at_25 0.3417688494651709
recall_rate_at_30 0.3524654317766762
recall_rate_at_50 0.4017740673102009
recall_rate_at_75 0.4294286459692147
recall_rate_at_10 0.44769110357422387


# Textually Dissimilar

In [21]:
textually_dissimilar_data = pd.read_csv("../Firefox/firefox_textually_dissimilar_duplicates.csv")

In [22]:
textually_dissimilar_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3924 entries, 0 to 3923
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Bug ID             3924 non-null   int64  
 1   Type               3924 non-null   object 
 2   Summary            3924 non-null   object 
 3   Product            3924 non-null   object 
 4   Component          3924 non-null   object 
 5   Status             3924 non-null   object 
 6   Resolution         3924 non-null   object 
 7   Changed            3924 non-null   object 
 8   Description        3924 non-null   object 
 9   Duplicate_Bug_Ids  3924 non-null   float64
dtypes: float64(1), int64(1), object(8)
memory usage: 306.7+ KB


In [23]:
sentence_dissim = textually_dissimilar_data.Description

tokens_dissim = []
for s in sentence_dissim:
    cleanedTex=re.sub(r'[^\w\s]','',str(s)).lower()
    words = (word_tokenize(cleanedTex))
    tokens_dissim.append(words)

In [28]:
count=0
dup_indices_with_bug_id_dissimilar = {}
similarity_scores_before = []
similarity_scores = {}

issues = data.Bug_Id

for i, row in textually_dissimilar_data.iterrows():
    
    issue_id = row["Bug ID"]
    dup_id = row.Duplicate_Bug_Ids
    
    if dup_id not in issues.values:
        count+=1
#         print(f'{i}. Duplicate ID {dup_id} not found in issue IDs')
        continue
    if i== 2292:
        break
    similarity_row = bm25.get_scores(tokens_dissim[i])
    argsort_similarity = np.argsort(similarity_row)[::-1][1:]
    similarity_scores_before = (np.sort(similarity_row)[::-1][1:])
#     similarity_scores[issue_id] = np.min(similarity_scores_before[np.nonzero(similarity_scores_before)])
    dup_bug_index = list(issues[issues == dup_id].index)[0]
    dup_ranking = np.argmax(argsort_similarity == np.int64(dup_bug_index))
    
    dup_indices_with_bug_id_dissimilar[issue_id] = dup_ranking
# print(count)

In [29]:
count_at_1=0
count_at_5=0
count_at_10=0
count_at_20=0
count_at_25=0
count_at_30=0
count_at_50=0
count_at_75=0
count_at_100=0

for key, value in enumerate(dup_indices_with_bug_id_dissimilar.values()):
    if(value < 1):
        count_at_1+=1
    if(value < 5):
        count_at_5+=1
    if(value < 10):
        count_at_10+=1
    if(value < 20):
        count_at_20+=1
    if(value < 25):
        count_at_25+=1
    if(value < 30):
        count_at_30+=1
    if(value < 50):
        count_at_50+=1
    if(value < 75):
        count_at_75+=1
    if(value < 100):
        count_at_100+=1
        
print("count at 1",count_at_1)        
print("count at 5",count_at_5)
print("count at 10",count_at_10)
print("count at 20",count_at_20)
print("count at 25",count_at_25)
print("count at 30",count_at_30)
print("count at 50",count_at_50)
print("count at 75",count_at_75)
print("count at 100",count_at_100)

N = len(dup_indices_with_bug_id_dissimilar)

recall_rate_at_1 = count_at_1/N
print("recall_rate_at_1",recall_rate_at_1)

recall_rate_at_5 = count_at_5/N
print("recall_rate_at_5",recall_rate_at_5)

recall_rate_at_10 = count_at_10/N
print("recall_rate_at_10",recall_rate_at_10)

recall_rate_at_20 = count_at_20/N
print("recall_rate_at_20",recall_rate_at_20)

recall_rate_at_25 = count_at_25/N
print("recall_rate_at_25",recall_rate_at_25)

recall_rate_at_30 = count_at_30/N
print("recall_rate_at_30",recall_rate_at_30)

recall_rate_at_50 = count_at_50/N
print("recall_rate_at_50",recall_rate_at_50)

recall_rate_at_75 = count_at_75/N
print("recall_rate_at_75",recall_rate_at_75)

recall_rate_at_100 = count_at_100/N
print("recall_rate_at_10",recall_rate_at_100)

count at 1 153
count at 5 373
count at 10 453
count at 20 549
count at 25 588
count at 30 624
count at 50 707
count at 75 784
count at 100 847
recall_rate_at_1 0.06766917293233082
recall_rate_at_5 0.16497125165855817
recall_rate_at_10 0.20035382574082264
recall_rate_at_20 0.24281291463954002
recall_rate_at_25 0.26006191950464397
recall_rate_at_30 0.27598407784166296
recall_rate_at_50 0.3126934984520124
recall_rate_at_75 0.34674922600619196
recall_rate_at_10 0.3746130030959752
